# NVDA Stock Article extractions

In [1]:
import requests

# api_key = '23f5208b7e2940aeb8b72c67195565b2'
api_key = 'bd9bb272078b41ebb0ee0a905d4a9532'
date_from = '2023-12-11'
date_to = '2024-01-08'
# Expanded list of sources
sources = 'bloomberg,cnbc,reuters,financial-times,techcrunch,the-wall-street-journal,the-verge,business-insider,the-economist,wired,engadget,bbc-news,fortune,techradar'

# Broadening the search query
query = ('NVDA OR "NVIDIA Corporation" OR "Graphics Processing Units" OR "Jensen Huang" '
         'OR "Artificial Intelligence" OR "Gaming Industry" OR "Computer Graphics" '
         'OR "NVIDIA products" OR "NVIDIA stock" OR "Semiconductor industry" '
         'OR "Deep learning" OR "GPU technology" OR "NVIDIA partnerships" '
         'OR "NVIDIA competition" OR "NVIDIA financials" OR "Tech industry" '
         'OR "Data centers" OR "Autonomous vehicles" OR "Gaming GPUs" '
         'OR "NVIDIA acquisitions" OR "Ray tracing technology"')

url = f'https://newsapi.org/v2/everything?q={query}&from={date_from}&to={date_to}&sources={sources}&apiKey={api_key}'
response = requests.get(url)
data = response.json()

# Check for 'articles' in response and extract headlines with publication dates
if 'articles' in data:
    news_data = [(article['title'], article['publishedAt'][:10]) for article in data['articles']]
    for headline, date in news_data:
        print(f"Date: {date}, Headline: {headline}")
else:
    print("Error or no articles found. Response:")
    print(data)


Date: 2024-01-04, Headline: Can artificial intelligence help you get fit?
Date: 2023-12-20, Headline: What Happens When Facebook Heats Your Home
Date: 2024-01-08, Headline: Staying One Step Ahead of Hackers When It Comes to AI
Date: 2023-12-25, Headline: The Hollywood Strikes Stopped AI From Taking Your Job. But for How Long?
Date: 2024-01-01, Headline: The top 10 people in artificial-intelligence healthcare
Date: 2023-12-25, Headline: The top 10 people in artificial-intelligence hardware
Date: 2023-12-27, Headline: The top 15 people in enterprise artificial intelligence
Date: 2023-12-20, Headline: The top 12 people in artificial-intelligence policy, ethics, and research
Date: 2023-12-22, Headline: The top 5 people in artificial-intelligence sustainability
Date: 2023-12-29, Headline: The top 7 people in artificial-intelligence e-commerce
Date: 2024-01-05, Headline: The top 7 people in artificial intelligence defense
Date: 2024-01-08, Headline: Waymo's driveless cars are hitting Phoenix

# Sentiment Analysis on News Articles

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('yiyanghkust/finbert-tone')
model = AutoModelForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone')

# Extract just the headlines from news_data
headlines = [headline for headline, _ in news_data]

# Tokenize and encode headlines
max_length = 512  # You can adjust this based on your requirements
inputs = tokenizer(headlines, padding=True, truncation=True, max_length=max_length, return_tensors="pt")

# Predict sentiment
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

# Pair each sentiment score with its corresponding date
sentiment_data = []
for (headline, date), prediction in zip(news_data, predictions):
    sentiment_data.append({
        'date': date,
        'headline': headline,
        'sentiment_positive': prediction[0].item(),
        'sentiment_neutral': prediction[1].item(),
        'sentiment_negative': prediction[2].item()
    })


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [3]:
sentiment_data

[{'date': '2024-01-04',
  'headline': 'Can artificial intelligence help you get fit?',
  'sentiment_positive': 0.8245240449905396,
  'sentiment_neutral': 0.15412722527980804,
  'sentiment_negative': 0.02134871855378151},
 {'date': '2023-12-20',
  'headline': 'What Happens When Facebook Heats Your Home',
  'sentiment_positive': 0.9603115916252136,
  'sentiment_neutral': 7.677675966988318e-06,
  'sentiment_negative': 0.039680760353803635},
 {'date': '2024-01-08',
  'headline': 'Staying One Step Ahead of Hackers When It Comes to AI',
  'sentiment_positive': 0.9951790571212769,
  'sentiment_neutral': 0.0030404203571379185,
  'sentiment_negative': 0.0017806445248425007},
 {'date': '2023-12-25',
  'headline': 'The Hollywood Strikes Stopped AI From Taking Your Job. But for How Long?',
  'sentiment_positive': 0.05843866616487503,
  'sentiment_neutral': 0.00040194709436036646,
  'sentiment_negative': 0.9411593675613403},
 {'date': '2024-01-01',
  'headline': 'The top 10 people in artificial-int

# Historical Stock Data and Sentiment Analysis Combined

In [4]:
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import StandardScaler

# Convert sentiment_data to a DataFrame
sentiment_df = pd.DataFrame(sentiment_data)

# Convert date to datetime and set as index
sentiment_df['date'] = pd.to_datetime(sentiment_df['date'])
sentiment_df.set_index('date', inplace=True)

# Aggregate sentiment scores by date
average_sentiment = sentiment_df.groupby('date').mean()

# Download stock data
ticker_symbol = 'NVDA'
stock_data = yf.download(ticker_symbol, start='2023-12-09', end='2024-01-08')

# Selecting the required columns
stock_data = stock_data[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]

# Convert stock_data index to datetime
stock_data.index = pd.to_datetime(stock_data.index)

# Combine stock data with sentiment data
combined_data = stock_data.join(average_sentiment)

# Interpolate missing values
combined_data = combined_data.interpolate(method='time')

# Scale price-related columns
price_cols = ['Open', 'High', 'Low', 'Close', 'Adj Close']
scaler_prices = StandardScaler()
combined_data[price_cols] = scaler_prices.fit_transform(combined_data[price_cols])

# Scale Volume column independently
scaler_volume = StandardScaler()
combined_data['Volume'] = scaler_volume.fit_transform(combined_data[['Volume']])

print(combined_data)

<ipython-input-4-254bafc85a9c>:13: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  average_sentiment = sentiment_df.groupby('date').mean()


[*********************100%%**********************]  1 of 1 completed
                Open      High       Low     Close  Adj Close    Volume  \
Date                                                                      
2023-12-11 -1.139163 -2.065162 -2.281585 -2.345860  -2.345860  1.699157   
2023-12-12 -2.599473 -1.895646 -2.060102 -1.154616  -1.154616  0.071428   
2023-12-13 -0.999701 -0.730382 -0.458460 -0.656146  -0.656146  0.965108   
2023-12-14 -0.230640 -0.634950 -0.649179 -0.353132  -0.353132  0.294774   
2023-12-15 -0.428716  0.286713  0.066537  0.271402   0.271402  1.340642   
2023-12-18  0.790060  1.578797  1.122676  1.644220   1.644220  0.547868   
2023-12-19  0.814313  0.658391  0.861206  1.097177   1.097177  1.162463   
2023-12-20  1.047760  1.033835  0.043978 -0.629548  -0.629548  0.373730   
2023-12-21  0.194819 -0.101289  0.373124  0.387056   0.387056 -0.781447   
2023-12-22  0.582890  0.260341  0.422344  0.202008   0.202008 -1.353720   
2023-12-26  0.353483  0.532823 

In [5]:
import pandas as pd
df = pd.DataFrame(combined_data)
df

,Open,High,Low,Close,Adj Close,Volume,sentiment_positive,sentiment_neutral,sentiment_negative
Date,,,,,,,,,
2023-12-11,-1.139163,-2.065162,-2.281585,-2.345860,-2.345860,1.699157,0.498930,4.999770e-01,1.092645e-03
2023-12-12,-2.599473,-1.895646,-2.060102,-1.154616,-1.154616,0.071428,0.499987,4.999998e-01,1.345586e-05
2023-12-13,-0.999701,-0.730382,-0.458460,-0.656146,-0.656146,0.965108,0.635214,3.148438e-02,3.333018e-01
2023-12-14,-0.230640,-0.634950,-0.649179,-0.353132,-0.353132,0.294774,0.807676,1.429543e-01,4.936961e-02
2023-12-15,-0.428716,0.286713,0.066537,0.271402,0.271402,1.340642,0.999564,3.488957e-04,8.674543e-05
2023-12-18,0.790060,1.578797,1.122676,1.644220,1.644220,0.547868,0.749933,9.270240e-07,2.500658e-01
2023-12-19,0.814313,0.658391,0.861206,1.097177,1.097177,1.162463,0.333106,3.333283e-01,3.335655e-01
2023-12-20,1.047760,1.033835,0.043978,-0.629548,-0.629548,0.373730,0.716802,1.533209e-01,1.298772e-01
2023-12-21,0.194819,-0.101289,0.373124,0.387056,0.387056,-0.781447,0.598839,3.999915e-01,1.169265e-03


# Transformer Architecture

In [10]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
from tensorflow.keras.callbacks import EarlyStopping

selected_features = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'sentiment_positive', 'sentiment_neutral', 'sentiment_negative']
data = combined_data[selected_features]

train_size = int(len(data) * 0.7)
val_size = int(len(data) * 0.15)
train_data = data.iloc[:train_size]
val_data = data.iloc[train_size:train_size + val_size]
test_data = data.iloc[train_size + val_size:]

# Prepare features and target for model training
X_train = train_data.drop('Close', axis=1)
y_train = train_data['Close']
X_val = val_data.drop('Close', axis=1)
y_val = val_data['Close']
X_test = test_data.drop('Close', axis=1)
y_test = test_data['Close']

# Define and compile the neural network model
model = Sequential()
model.add(Dense(32, activation='LeakyReLU', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(16, activation='LeakyReLU'))
model.add(Dense(1))
model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(
    X_train, y_train,
    epochs=50,  # Set back to 50 or an arbitrarily large number
    batch_size=8,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping]  # Add the early stopping callback here
)

train_loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(train_loss) + 1)

# Convert the epochs range object to a list
epochs_list = list(epochs)

# Plotting the training and validation loss
loss_fig = go.Figure()
loss_fig.add_trace(go.Scatter(x=epochs_list, y=train_loss, mode='lines', name='Training Loss'))
loss_fig.add_trace(go.Scatter(x=epochs_list, y=val_loss, mode='lines', name='Validation Loss'))
loss_fig.update_layout(title='Training and Validation Loss per Epoch',
                       xaxis_title='Epochs',
                       yaxis_title='Loss',
                       legend_title='Type')
loss_fig.show()

# Make predictions
predictions = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
print(f'MSE: {mse}, MAE: {mae}, R^2: {r2}')

# Reshape y_test and predictions for inverse transform
y_test_reshaped = y_test.values.reshape(-1, 1)
predictions_reshaped = predictions.reshape(-1, 1)

num_scaled_cols = 5

# Create separate dummy arrays for inverse scaling
dummy_array_y_test = np.zeros((len(y_test_reshaped), num_scaled_cols))
dummy_array_predictions = np.zeros((len(predictions_reshaped), num_scaled_cols))

# Fill in the 'Close' column values in the dummy arrays
# Assuming 'Close' is the last of the scaled columns
dummy_array_y_test[:, -1] = y_test_reshaped.flatten()
dummy_array_predictions[:, -1] = predictions_reshaped.flatten()

# Inverse transform the 'Close' prices using the dummy arrays
y_test_original = scaler_prices.inverse_transform(dummy_array_y_test)[:, -1]
predictions_original = scaler_prices.inverse_transform(dummy_array_predictions)[:, -1]

# Extracting testing dates
test_dates = test_data.index

# Plotting with Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=test_dates, y=y_test_original, mode='lines', name='Actual'))
fig.add_trace(go.Scatter(x=test_dates, y=predictions_original, mode='lines', name='Predicted'))
fig.update_layout(title='Actual vs Predicted Stock Prices', xaxis_title='Date', yaxis_title='Stock Price', legend_title='Legend')
fig.show()

Epoch 1/50
2/2 [==============================] - 1s 161ms/step - loss: 1.0373 - val_loss: 0.1579
Epoch 2/50
2/2 [==============================] - 0s 29ms/step - loss: 0.4119 - val_loss: 0.4172
Epoch 3/50
2/2 [==============================] - 0s 30ms/step - loss: 0.1760 - val_loss: 0.3774
Epoch 4/50
2/2 [==============================] - 0s 38ms/step - loss: 0.2332 - val_loss: 0.1166
Epoch 5/50
2/2 [==============================] - 0s 40ms/step - loss: 0.1781 - val_loss: 0.0326
Epoch 6/50
2/2 [==============================] - 0s 29ms/step - loss: 0.1636 - val_loss: 0.0883
Epoch 7/50
2/2 [==============================] - 0s 29ms/step - loss: 0.0889 - val_loss: 0.1026
Epoch 8/50
2/2 [==============================] - 0s 33ms/step - loss: 0.1249 - val_loss: 0.0679
Epoch 9/50
2/2 [==============================] - 0s 30ms/step - loss: 0.0760 - val_loss: 0.0713
Epoch 10/50
2/2 [==============================] - 0s 35ms/step - loss: 0.0935 - val_loss: 0.1166
Epoch 10: early stopping


1/1 [==============================] - 0s 56ms/step
MSE: 0.01329717416131488, MAE: 0.10339301756045549, R^2: 0.9680438296110395


In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 833 (3.25 KB)
Trainable params: 833 (3.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
